In [ ]:
#default_exp expanding

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

# Expanding

> Expanding window operations

In [ ]:
#hide
import random

from nbdev.showdoc import *

In [ ]:
#export
from math import sqrt
from typing import Callable, Optional

import numpy as np
from numba import njit

from window_ops.rolling import *
from window_ops.utils import _gt, _lt

In [ ]:
import pandas as pd

np.random.seed(0)
y = np.random.rand(100)
ys = pd.Series(y)

In [ ]:
#exporti
def _expanding_op(rolling_op: Callable, x: np.ndarray, min_samples: int = 1) -> np.ndarray:
    n_samples = x.size
    return rolling_op(x, window_size=n_samples, min_samples=min_samples)

In [ ]:
#export
def expanding_mean(x: np.ndarray) -> np.ndarray:
    return _expanding_op(rolling_mean, x)

In [ ]:
assert np.allclose(expanding_mean(y), ys.expanding().mean().values)

In [ ]:
#export
def expanding_std(x: np.ndarray) -> np.ndarray:
    return _expanding_op(rolling_std, x, min_samples = 2)

In [ ]:
assert np.allclose(expanding_std(y), ys.expanding().std().values, equal_nan=True)

In [ ]:
#exporti
@njit
def _expanding_comp(comp: Callable,
                    x: np.ndarray) -> np.ndarray:
    n_samples = x.size
    out = np.empty(n_samples, dtype=np.float32)
    pivot = x[0]
    out[0] = pivot
    for i in range(1, n_samples):
        if comp(x[i], pivot) > 0:
            pivot = x[i]
        out[i] = pivot
    return out

In [ ]:
#export
def expanding_max(x: np.ndarray) -> np.ndarray:
    return _expanding_comp(_gt, x)

In [ ]:
assert np.allclose(expanding_max(y), ys.expanding().max().values)

In [ ]:
#export
def expanding_min(x: np.ndarray) -> np.ndarray:
    return _expanding_comp(_lt, x)

In [ ]:
assert np.allclose(expanding_min(y), ys.expanding().min().values)

## Seasonal

In [ ]:
y_df = ys.to_frame('y')
y_df['season'] = np.arange(7)[[i % 7 for i in range(y.size)]]

In [ ]:
#exporti
def _seasonal_expanding_op(rolling_op: Callable, x: np.ndarray, season_length: int,
                           min_samples: int = 1) -> np.ndarray:
    n_samples = x.size
    return rolling_op(x, season_length=season_length, window_size=n_samples, min_samples=min_samples)

In [ ]:
#export
def seasonal_expanding_mean(x: np.ndarray, season_length: int) -> np.ndarray:
    return _seasonal_expanding_op(seasonal_rolling_mean, x, season_length)

In [ ]:
assert np.allclose(y_df.groupby('season')['y'].transform(lambda y: y.expanding().mean()).values,
                   seasonal_expanding_mean(y, 7))

In [ ]:
#export
def seasonal_expanding_std(x: np.ndarray, season_length: int) -> np.ndarray:
    return _seasonal_expanding_op(seasonal_rolling_std, x, season_length, min_samples=2)
    return out

In [ ]:
assert np.allclose(y_df.groupby('season')['y'].transform(lambda y: y.expanding().std()).values,
                   seasonal_expanding_std(y, 7),
                   equal_nan=True)

In [ ]:
#export
def seasonal_expanding_min(x: np.ndarray, season_length: int):
    return _seasonal_expanding_op(seasonal_rolling_min, x, season_length)

In [ ]:
assert np.allclose(y_df.groupby('season')['y'].transform(lambda y: y.expanding().min()).values,
                   seasonal_expanding_min(y, 7))

In [ ]:
#export
def seasonal_expanding_max(x: np.ndarray, season_length: int):
    return _seasonal_expanding_op(seasonal_rolling_max, x, season_length)

In [ ]:
assert np.allclose(y_df.groupby('season')['y'].transform(lambda y: y.expanding().max()).values,
                   seasonal_expanding_max(y, 7))